In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p1/submit.csv
/kaggle/input/2022-ai-w4p1/train.csv
/kaggle/input/2022-ai-w4p1/test.csv


In [2]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('/kaggle/input/2022-ai-w4p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w4p1/test.csv')
submit = pd.read_csv('/kaggle/input/2022-ai-w4p1/submit.csv')

In [4]:
train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
0,1,95,60,18,58,23.9,0.260,22,0
1,5,105,72,29,325,36.9,0.159,28,0
2,0,135,68,42,250,42.3,0.365,24,1
3,4,131,68,21,166,33.1,0.160,28,0
4,1,103,30,38,83,43.3,0.183,33,0
...,...,...,...,...,...,...,...,...,...
532,5,139,64,35,140,28.6,0.411,26,0
533,1,96,122,0,0,22.4,0.207,27,0
534,10,101,86,37,0,45.6,1.136,38,1
535,0,141,0,0,0,42.4,0.205,29,1


In [5]:
train.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Diabetes'],
      dtype='object')

In [6]:
x_train = np.array(train[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']])
y_train = np.array(train[['Diabetes']])
x_test = np.array(test)

In [7]:
from sklearn.preprocessing import RobustScaler
trans1 = RobustScaler().fit(x_train)
x_train = trans1.transform(x_train)
trans2 = RobustScaler().fit(x_test)
x_test = trans2.transform(x_test)

In [8]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)

In [9]:
loss = torch.nn.BCELoss()

W = torch.zeros((8,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.001)

nb_epochs = 10000

In [10]:
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    cost = loss(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693147
Epoch  100/10000 Cost: 0.685444
Epoch  200/10000 Cost: 0.678116
Epoch  300/10000 Cost: 0.671141
Epoch  400/10000 Cost: 0.664497
Epoch  500/10000 Cost: 0.658165
Epoch  600/10000 Cost: 0.652126
Epoch  700/10000 Cost: 0.646363
Epoch  800/10000 Cost: 0.640859
Epoch  900/10000 Cost: 0.635600
Epoch 1000/10000 Cost: 0.630570
Epoch 1100/10000 Cost: 0.625756
Epoch 1200/10000 Cost: 0.621145
Epoch 1300/10000 Cost: 0.616727
Epoch 1400/10000 Cost: 0.612490
Epoch 1500/10000 Cost: 0.608424
Epoch 1600/10000 Cost: 0.604520
Epoch 1700/10000 Cost: 0.600768
Epoch 1800/10000 Cost: 0.597161
Epoch 1900/10000 Cost: 0.593691
Epoch 2000/10000 Cost: 0.590350
Epoch 2100/10000 Cost: 0.587132
Epoch 2200/10000 Cost: 0.584031
Epoch 2300/10000 Cost: 0.581041
Epoch 2400/10000 Cost: 0.578155
Epoch 2500/10000 Cost: 0.575370
Epoch 2600/10000 Cost: 0.572680
Epoch 2700/10000 Cost: 0.570081
Epoch 2800/10000 Cost: 0.567568
Epoch 2900/10000 Cost: 0.565138
Epoch 3000/10000 Cost: 0.562786
Epoch 31

In [11]:
hypothesis = torch.sigmoid(x_test.matmul(W)+b)
prediction = hypothesis >= torch.FloatTensor([0.5])

In [12]:
submit['Diabetes'] = prediction
submission=submit.astype(np.int32)
submission.to_csv('submission.csv',mode='w',header=True,index=False)